In [ ]:
import csv
import sys
import pandas as pd
import numpy as np

# maxInt = sys.maxsize
# while True:
#     # decrease the maxInt value by factor 10 
#     # as long as the OverflowError occurs.
#     try:
#         csv.field_size_limit(maxInt)
#         break
#     except OverflowError:
#         maxInt = int(maxInt/10)


chunk_size = 300

csv_file_path = 'articles2.csv'

documents = []
metadatas = []
ids = []
id = 1

# def process_chunk(chunk):
#     id = 1

#     for i, line in chunk.iterrows():
#         if i==0:
#             continue
#         documents.append(df.iloc[9])
#         metadatas.append
#         ({
#             "id": df.iloc[1],
#             "title": df.iloc[2],
#             "publication": df.iloc[3],
#             "author": df.iloc[4],
#             "date": df.iloc[5],
#             "year": df.iloc[6],
#             "month": df.iloc[7]
#         })
#         ids.append(str(id))
#         id+=1

#         return documents, metadatas, ids


# all_documents = []
# all_metadatas = []
# all_ids = []

for chunk in pd.read_csv(csv_file_path, encoding='utf8', chunksize=chunk_size):
    documents.extend(chunk.iloc[:, 9:].astype(str).values.flatten())
    #metadata_chunk=({"id": chunk.iloc[1],"title": chunk.iloc[2],"publication": chunk.iloc[3],"author": chunk.iloc[4],"date": chunk.iloc[5],"year": chunk.iloc[6],"month": chunk.iloc[7]})
    #documents.append(chunk.iloc[9].values)
    metadata_chunk = chunk.iloc[:, 1:8].to_dict(orient='records')
    metadatas.extend(metadata_chunk)
    # metadatas.append
    # ({
    #     "id": chunk.iloc[1],
    #     "title": chunk.iloc[2],
    #     "publication": chunk.iloc[3],
    #     "author": chunk.iloc[4],
    #     "date": chunk.iloc[5],
    #     "year": chunk.iloc[6],
    #     "month": chunk.iloc[7]
    # })
    ids.extend(chunk.index.astype(str))
    #id+=1
    # documents, metadatas, ids = process_chunk(chunk)

    # all_documents.extend(documents)
    # all_metadatas.extend(metadatas)
    # all_ids.extend(ids)

In [ ]:
import pandas as pd

csv_file_path = 'articles1.csv'
chunk_size = 20

documents = []
metadatas = []
ids = []
id = 1
chunks=0

def upload (documents, metadatas, ids):
    collection.add(documents=documents, metadatas=metadatas, ids=ids)

for chunk in pd.read_csv(csv_file_path, encoding='utf8', chunksize=chunk_size):
    
    documents.extend(chunk.iloc[:, 9])
    metadata_chunk = chunk.iloc[:, 1:8].to_dict(orient='records')
    metadatas.extend(metadata_chunk)
    ids.extend(chunk.index.astype(str))
    upload(documents, metadatas, ids)
    chunks+=1

In [ ]:
import chromadb
import csv
import sys
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="my_collection")

maxInt = sys.maxsize
while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

# Initialize arrays to store data
metadatas = []
documents = []
ids = []
id = 1

# Replace 'your_file.csv' with the actual path to your CSV file
csv_file_path = 'articles1.csv'

def upload (documents, metadatas, ids):
    collection.add(documents=documents, metadatas=metadatas, ids=ids)

# Open the CSV file
with open(csv_file_path, 'r', encoding='utf8') as csv_file:
    # Create a CSV reader object
    csv_reader = csv.reader(csv_file)

    # Skip the header if it exists
    next(csv_reader, None)

    # Iterate through each row in the CSV file
    for id_counter, row in enumerate(csv_reader, start = 1):
        # Extract columns 1-8 into a dictionary and append to metadatas
        metadatas.append({
            'id': row[1],
            'title': row[2],
            'publication': row[3],
            'author': row[4],
            'date': row[5],
            'year': row[6],
            'month': row[7],
            'url': row[8]
            })
        ids.append(str(id))
        id +=1

        # Extract column 9, split by newline, and append to documents
        documents.extend(row[9].split('\n'))
        upload(documents, metadatas, ids)



In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="my_collection")

df=pd.read_csv('articles1.csv', encoding='utf8')

csv_file_path = 'articles1.csv'
documents = []
metadatas = []
ids = []
id = 1

def upload (documents, metadatas, ids):
    collection.add(documents=documents, metadatas=metadatas, ids=ids)

chunk_size = 300
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
)

# Function to upload chunks to ChromaDB
def upload(documents, metadatas, ids):
    for i, document in enumerate(documents):
        metadata = metadatas[i]
        chunk = text_splitter.create_documents([documents], metadatas=[metadata])
        for j, sub_chunk in enumerate(chunk):
            collection.add({"chunk_id": j, "content": sub_chunk, "source": metadata["source"]})

# Read CSV file in chunks
for chunk in pd.read_csv(csv_file_path, encoding='utf8', chunksize=chunk_size):
    documents = chunk.iloc[:, 9].tolist()
    metadata_chunk = chunk.iloc[:, 1:8].to_dict(orient='records')
    metadatas = metadata_chunk
    ids = chunk.index.astype(str).tolist()
    upload(documents, metadatas, ids)





In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_name = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCasualLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
    trust_remote_code=True,
    )

In [ ]:
import chromadb
import csv
import sys
from nltk.tokenize import sent_tokenize

documents = []
metadatas = []
ids = []

chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="my_collection")

maxInt = sys.maxsize
while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

def upload (documents, metadatas, ids):
    collection.add(documents=documents, metadatas=metadatas, ids=ids)

def chunk_section(filename):
    id = 1
    with open(filename, 'r', encoding='utf8') as csvfile:
        datareader = csv.DictReader(csvfile)
        for row in datareader:
            #chunk "content" row by sentence
            lore = row["Lore"]
            sentences = sent_tokenize(lore)
            for sentence in sentences:
                #remove "content" from metadata dicitonary
                meta = {key: value for key, value in row.items() if key != 'Lore'}
                documents.append(sentence)
                metadatas.append(meta)
                ids.append(str(id))
                upload([sentence], [meta], [str(id)])
                id +=1
        